In [1]:
from flask import Flask, render_template, Response
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import mysql.connector

In [3]:
try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="attendance_db",
        use_pure=True
    )
    cursor = conn.cursor()
    print("Connected to MySQL database")
except mysql.connector.Error as e:
    print("Error while connecting to MySQL", e)

Connected to MySQL database


In [5]:
path = 'datawajah'

images = []
classNames = []
mylist = os.listdir(path)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [7]:
def findEncodings(images):
    encodeList = []
    for img in images:
        if img is None:  # Check if the image is loaded correctly
            print("Empty image encountered!")
            continue  # Skip this image if it is empty
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
    return encodeList
encoded_face_train = findEncodings(images)

Empty image encountered!
Empty image encountered!


In [9]:
def markAttendance(name):
    cursor.execute("SELECT * FROM attendance WHERE name = %s AND date = CURDATE()", (name,))
    result = cursor.fetchone()
    if result is None:
        now = datetime.now()
        time = now.strftime('%H:%M:%S')
        date = now.strftime('%Y-%m-%d')
        cursor.execute("INSERT INTO attendance (name, time, date) VALUES (%s, %s, %s)", (name, time, date))
        conn.commit()

In [11]:
# Video feed generator function
def generate_frames():
    cap = cv2.VideoCapture(0)
    while True:
        success, img = cap.read()
        if not success:
            print("Failed to capture image")
            break
        else:
            imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
            faces_in_frame = face_recognition.face_locations(imgS)
            encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
            
            for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
                matches = face_recognition.compare_faces(encoded_face_train, encode_face)
                faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
                matchIndex = np.argmin(faceDist)
                
                if matches[matchIndex]:
                    name = classNames[matchIndex].upper()
                    markAttendance(name)
                    
                    # Display the detected face box
                    y1, x2, y2, x1 = faceloc
                    y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            ret, buffer = cv2.imencode('.jpg', img)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
    cap.release()

In [13]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('home.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Oct/2024 16:01:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2024 16:01:10] "GET /video_feed HTTP/1.1" 200 -
